## neo4j 사용 예시

In [1]:
!pip install neo4j

     |████████████████████████████████| 89 kB 9.5 MB/s  eta 0:00:01
  Created wheel for neo4j: filename=neo4j-4.4.0-py3-none-any.whl size=114861 sha256=a6e1844328feb34628666527d9cd1f8ef865760ba3a18dd6ae70e8e7187a807b
  Stored in directory: /Users/datawhales/Library/Caches/pip/wheels/ae/a8/53/9c108d3af55c886b03a3f088fb3d79345e82452fe525eed3f0
Successfully built neo4j


In [2]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://100.24.23.207:7687'
user = 'neo4j'
password = 'patient-tolerances-grams'
driver = GraphDatabase.driver(host,auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [6]:
author = 'ABC'
title = 'ABC books'

neo4j_query("""
MERGE (a:Author{name:$author})
MERGE (b:Article{title:$title})
MERGE (a)-[:WROTE]->(b)
""", {'title':title, 'author':author})

""


## triple2graph

In [21]:
def reltriple2graph(subj_name, subj_type, obj_name, obj_type, relation):
    query_string = 'MERGE (a:' + subj_type + '{label:$subj_name})' + \
                      'MERGE (b:' + obj_type + '{label:$obj_name})' + \
                      'MERGE (a)-[:' + relation + ']->(b)'
    query_info_dict = {'subj_name': subj_name, 'obj_name': obj_name}
    
    neo4j_query(query_string, query_info_dict)
    return query_string

In [23]:
subj_name = '갤럭시 플립2'
subj_type = 'ARTIFACT'
obj_name = '삼성전자'
obj_type = 'ORGANIZATION'
relation = '해당_제품의_제조사'

reltriple2graph(subj_name, subj_type, obj_name, obj_type, relation)

'MERGE (a:ARTIFACT{label:$subj_name})MERGE (b:ORGANIZATION{label:$obj_name})MERGE (a)-[:해당_제품의_제조사]->(b)'

## 추출된 triple data 전처리

In [24]:
import pandas as pd

df = pd.read_csv('../results/specific_domain_pred_true.csv')

In [25]:
df.head()

,sentence,subj_name,subj_type,obj_name,obj_type,relation
0,"[E1] 삼성전자 [/E1] 가 [E2] 인도 [/E2] , 중국 시장을 겨냥해 ...",삼성전자,ORGANIZATION,인도,COUNTRY,다음 국가의 것임(country)
1,"[E1] 삼성전자 [/E1] 가 인도, [E2] 중국 [/E2] 시장을 겨냥해 ...",삼성전자,ORGANIZATION,중국,COUNTRY,다음 국가의 것임(country)
2,"[E1] 삼성전자 [/E1] 가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 ...",삼성전자,ORGANIZATION,스마트폰,TERM,해당 개체의 제품(product or material produced)
3,"삼성전자가 [E1] 인도 [/E1] , [E2] 중국 [/E2] 시장을 겨냥해...",인도,COUNTRY,중국,COUNTRY,수교국(diplomatic relation)
4,"삼성전자가 [E1] 인도 [/E1] , 중국 시장을 겨냥해 혁신기술을 넣은 중저가...",인도,COUNTRY,스마트폰,TERM,해당 개체의 제품(product or material produced)


In [34]:
df.relation.value_counts()

해당 개체의 배우자(spouse)                                           4601
해당 개체가 다음으로 이루어져 있음(has part)                                3240
다음 공동체의 구성원임(member of)                                      2515
다음 국가의 것임(country)                                           2268
다음 행정구역에 위치함(located in)                                     2074
                                                             ... 
해당 장소나 사건에서 다음 언어가 주로 사용됨(language used)                        4
해당 지역에 대해 영유권 또는 관할권을 주장하는 국가·지역·단체(territory claimed by)       3
해당 주제의 상위 주제(facet of)                                          3
반대 속성(inverse property)                                         2
위키데이터 속성(Wikidata property)                                     1
Name: relation, Length: 90, dtype: int64

In [41]:
import re

def convert_relation(rel_string):
    """
    Example:
        rel_string = '다음 국가의 것임(country)'

    Return:
        '다음_국가의_것임'
    """
    # 괄호 제거
    rel_string = re.sub(r'\([^)]*\)','', rel_string)
    
    rel_string = re.sub('·', '_', rel_string)
    
    string_list = rel_string.split()
    
    return '_'.join(string_list)

In [42]:
convert_relation('다음 국가의 것임(country)')

'다음_국가의_것임'

In [46]:
convert_relation('해당 개체의 제품(product or material produced)')

'해당_개체의_제품'

In [47]:
df['converted_relation'] = df['relation'].apply(lambda x: convert_relation(x))

In [50]:
df.head()

,sentence,subj_name,subj_type,obj_name,obj_type,relation,converted_relation
0,"[E1] 삼성전자 [/E1] 가 [E2] 인도 [/E2] , 중국 시장을 겨냥해 ...",삼성전자,ORGANIZATION,인도,COUNTRY,다음 국가의 것임(country),다음_국가의_것임
1,"[E1] 삼성전자 [/E1] 가 인도, [E2] 중국 [/E2] 시장을 겨냥해 ...",삼성전자,ORGANIZATION,중국,COUNTRY,다음 국가의 것임(country),다음_국가의_것임
2,"[E1] 삼성전자 [/E1] 가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 ...",삼성전자,ORGANIZATION,스마트폰,TERM,해당 개체의 제품(product or material produced),해당_개체의_제품
3,"삼성전자가 [E1] 인도 [/E1] , [E2] 중국 [/E2] 시장을 겨냥해...",인도,COUNTRY,중국,COUNTRY,수교국(diplomatic relation),수교국
4,"삼성전자가 [E1] 인도 [/E1] , 중국 시장을 겨냥해 혁신기술을 넣은 중저가...",인도,COUNTRY,스마트폰,TERM,해당 개체의 제품(product or material produced),해당_개체의_제품


In [49]:
from tqdm import tqdm

for i in tqdm(range(len(df))):
    tmp_row = df.iloc[i]
    
    subj_name, obj_name = tmp_row['subj_name'], tmp_row['subj_type']
    obj_name, obj_type = tmp_row['obj_name'], tmp_row['obj_type']
    relation = tmp_row['converted_relation']
    
    reltriple2graph(subj_name, subj_type, obj_name, obj_type, relation)

100%|██████████| 42006/42006 [2:31:06<00:00,  4.63it/s]  
